In [67]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from selenium.common.exceptions import TimeoutException
import random
import re

In [72]:
def get_district_details(district_name):
    service = Service(executable_path=r'/usr/bin/chromedriver')
    options = webdriver.ChromeOptions()
    options.add_argument('--headless') #It allows users to run automated scripts in headless mode, meaning that the browser window wouldn’t be visible. 
    driver = webdriver.Chrome(service=service, options=options)
    url = 'https://google.com'#"https://realt.by/sale/dachi/?page=1"
    headers = ({'User-Agent':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:124.0) Gecko/20100101 Firefox/124.0', 'Accept-Language':'en-US, en;q=0.5'})
    driver.get(url);
    time.sleep(1)
    #input_box = WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "textarea")))
    input_box = driver.find_element(By.CSS_SELECTOR, "textarea")
    input_box.send_keys(district_name)
    input_box.send_keys(Keys.ENTER)
    time.sleep(1)

    wiki_link = driver.find_element(By.XPATH,"//a[contains(@href, 'https://ru.wikipedia.org')]")
    wiki_link.click()

    def get_soup(page):
        soup = BeautifulSoup(page, 'html.parser')
        res = soup.find("span", class_='coordinates plainlinks nourlexpansion').find("a").text
        res = re.findall(r"(\d{2}\°\d{2,}\′*\d*).", res)
        res = [x.replace('°','.') for x in res]
        res = [x.replace('′','') for x in res]
        return (res[0],res[1])
    
    res_1 = get_soup(driver.page_source)

    try:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        res = soup.find_all('tr')
        for i in res:
            if i.find('th') and i.find('th').text == 'Адм. центр':
                wiki = i.find('a').text
        wiki_link2 = driver.find_element(By.LINK_TEXT, wiki)
        wiki_link2.click()
        res_2 = get_soup(driver.page_source)
    except:
        res_2 = res_1
        

    driver.quit()
    
    return [(float(res_1[0]), float(res_1[1])), (float(res_2[0]), float(res_2[1]))]

#res = get_district_details('г.Минск')
#res




In [69]:

page = requests.get('https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D0%BC%D0%B5%D0%BB%D1%8C%D1%81%D0%BA%D0%B8%D0%B9_%D1%80%D0%B0%D0%B9%D0%BE%D0%BD')
soup = BeautifulSoup(page.content, 'html.parser')
#res = soup.find('table')
res = soup.find_all('tr')
for i in res:
    if i.find('th') and i.find('th').text == 'Адм. центр':
        wiki_link2 = i.find('a').text